In [1]:
# @title Installations {"display-mode":"both"}
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [2]:

# @title imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import gradio as gr
from textblob import TextBlob
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
import xgboost as xgb
from xgboost import XGBClassifier
import logging
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, accuracy_score
import plotly.graph_objects as go
from datetime import datetime
import os

In [3]:
# @title graphs
# Define individual plotting functions
def plot_age_distribution(data):
    fig = px.histogram(data, x='Age', nbins=20, title="Distribution of Age of Candidates", color_discrete_sequence=["#6abcb0"])
    fig.update_layout(xaxis_title="Age", yaxis_title="Frequency")
    return fig

def plot_gender_distribution(data):
    fig = px.histogram(data, x='Sex', title='Gender Distribution of Candidates', color_discrete_sequence=['maroon'])
    fig.update_layout(xaxis_title='Gender', yaxis_title='Number of Candidates')
    return fig

def plot_vote_share_distribution(data):
    plt.figure(figsize=(12, 6))
    sns.boxplot(data=data, x='Year', y='Vote_Share_Percentage', palette='cubehelix', hue='Year', dodge=False)
    plt.title("Distribution of Vote Shares Over the Years")
    plt.xlabel("Year")
    plt.xticks(rotation=45)
    plt.ylabel("Vote Share (%)")
    plt.grid(axis='y')
    return plt.gcf()

def plot_vote_share_for_incumbents(data):
    incumbent_data = data[data['Incumbent'] == True]
    avg_vote_share_by_year = incumbent_data.groupby('Year')['Vote_Share_Percentage'].mean().reset_index()
    plt.figure(figsize=(10, 6))
    sns.lineplot(x='Year', y='Vote_Share_Percentage', data=avg_vote_share_by_year, marker='o', color='teal')
    plt.title('Average Vote Share for Incumbents Over Time')
    plt.xlabel('Year')
    plt.ylabel('Average Vote Share Percentage')
    return plt.gcf()

def plot_top_10_parties_vote_share(data):
    party_vote_share = data.groupby('Party')['Vote_Share_Percentage'].sum()
    top_10_parties = party_vote_share.nlargest(10)
    fig = px.pie(values=top_10_parties, names=top_10_parties.index, title='Top 10 Parties by Vote Share Percentage')
    return fig

def plot_vote_share_by_incumbency(data):
    plt.figure(figsize=(10, 6))
    sns.kdeplot(data=data, x='Vote_Share_Percentage', hue='Incumbent', fill=True)
    plt.title("Vote Share Distribution by Incumbency Status")
    return plt.gcf()

def plot_vote_margins_by_party(data):
    top_parties = data['Party'].value_counts().index[:5]
    filtered_data = data[data['Party'].isin(top_parties)]
    plt.figure(figsize=(12, 6))
    sns.violinplot(x='Party', y='Margin', hue='Party', data=filtered_data, palette="Set3", inner="quartile", dodge=False)
    plt.title('Vote Margins by Top 5 Parties')
    plt.xlabel('Party')
    plt.ylabel('Vote Margin')
    return plt.gcf()

def plot_success_rate_trend(data):
    winners = data[data['Position'] == 1]
    top_3_parties_by_wins = winners['Party'].value_counts().head(3)
    success_rate = data.groupby(['Party', 'Year'])['Position'].apply(lambda x: (x == 1).sum() / len(x)).reset_index()
    success_rate.columns = ['Party', 'Year', 'Success_Rate']
    success_rate_top_3_parties = success_rate[success_rate['Party'].isin(top_3_parties_by_wins.index)]
    plt.figure(figsize=(10, 6))
    sns.lineplot(x='Year', y='Success_Rate', hue='Party', data=success_rate_top_3_parties, palette='plasma')
    plt.title('Success Rate Trend Over Time for Top 3 Parties')
    plt.xlabel('Year')
    plt.ylabel('Success Rate')
    return plt.gcf()

def plot_candidate_type_by_party(data):
    top_parties = data['Party'].value_counts().index[:5]
    filtered_data = data[data['Party'].isin(top_parties)]
    plt.figure(figsize=(12, 6))
    sns.countplot(x='Party', hue='Candidate_Type', data=filtered_data, palette="Set2")
    plt.title('Party Representation by Candidate Type')
    plt.tight_layout()
    return plt.gcf()

def plot_vote_share_vs_age(data):
    top_parties = data['Party'].value_counts().index[:5]
    filtered_data = data[data['Party'].isin(top_parties)]
    plt.figure(figsize=(10, 6))
    sns.scatterplot(data=filtered_data, x='Age', y='Vote_Share_Percentage', hue='Party', alpha=0.6)
    plt.title('Scatter Plot of Vote Share Percentage vs Age by Party')
    plt.xlabel('Age')
    plt.ylabel('Vote Share Percentage')
    return plt.gcf()

def plot_candidates_by_party(data):
    top_parties = data['Party'].value_counts().index[:5]
    filtered_data = data[data['Party'].isin(top_parties)]
    plt.figure(figsize=(10, 6))
    sns.countplot(data=filtered_data, x='Party', order=filtered_data['Party'].value_counts().index, palette='YlOrBr')
    plt.title('Candidates by Party')
    plt.xlabel('Party')
    plt.ylabel('Number of Candidates')
    return plt.gcf()

def plot_margin_percentage_vs_age(data):
    top_parties = data['Party'].value_counts().index[:5]
    filtered_data = data[data['Party'].isin(top_parties)]
    plt.figure(figsize=(10, 6))
    sns.regplot(data=filtered_data, x='Age', y='Margin_Percentage', scatter_kws={'alpha': 0.3}, line_kws={'color': 'green'})
    plt.title('Regression Plot of Margin Percentage vs Age')
    plt.xlabel('Age')
    plt.ylabel('Margin Percentage')
    return plt.gcf()

def plot_pairplot_election_metrics(data):
    top_parties = data['Party'].value_counts().index[:5]
    filtered_data = data[data['Party'].isin(top_parties)]
    sns.pairplot(filtered_data, vars=['Votes', 'Vote_Share_Percentage', 'Margin', 'Margin_Percentage'], hue='Party', corner=True)
    plt.suptitle('Pair Plot of Key Election Metrics', y=1.02)
    return plt.gcf()

def plot_votes_vs_margin(data):
    top_parties = data['Party'].value_counts().index[:5]
    filtered_data = data[data['Party'].isin(top_parties)]
    sns.jointplot(data=filtered_data, x='Votes', y='Margin', kind='reg')
    plt.suptitle('Joint Plot of Votes vs Margin', y=1.02)
    return plt.gcf()

# Function to load data and create visualizations
def load_data_and_visualize(file):
    data = pd.read_csv(file.name)

    # Create plots and return them in a list
    plots = [
        plot_age_distribution(data),
        plot_gender_distribution(data),
        plot_vote_share_distribution(data),
        plot_vote_share_for_incumbents(data),
        plot_top_10_parties_vote_share(data),
        plot_vote_share_by_incumbency(data),
        plot_vote_margins_by_party(data),
        plot_success_rate_trend(data),
        plot_candidate_type_by_party(data),
        plot_vote_share_vs_age(data),
        plot_candidates_by_party(data),
        plot_margin_percentage_vs_age(data),
        plot_pairplot_election_metrics(data),
        plot_votes_vs_margin(data)
    ]

    return plots

# Gradio app
with gr.Blocks() as app:
    file_input = gr.File(label="Upload CSV File")
    gr.Markdown("## Election Data Visualizations")

    # Output container for the 14 plots
    plot_outputs = [gr.Plot(label=f"Plot {i+1}") for i in range(14)]

    # Button to load and visualize the data
    load_button = gr.Button("Load and Visualize")

    # When the button is clicked, load data and display plots
    load_button.click(
        fn=load_data_and_visualize,
        inputs=file_input,
        outputs=plot_outputs
    )

app.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7df90fb3aa8354156f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [15]:
# Load combined data outside the function
data = pd.read_csv('/content/combined_cleaned_data (1).csv')



# Preprocess data to add sentiment scores and party columns
data['sentiment_score'] = data['cleaned_text'].apply(lambda text: TextBlob(str(text)).sentiment.polarity)
parties = ['bjp', 'shiv sena', 'shiv sena ubt', 'congress', 'ncp']
for party in parties:
    data[party] = data['cleaned_text'].astype(str).apply(lambda x: party in x)

# Function to perform analysis and return outputs
def sentiment_analysis_and_visualization(sample_size=1000):
    sampled_data = data.sample(n=sample_size) if len(data) > sample_size else data

    # Calculate average sentiment scores for each party
    party_sentiments = {party: sampled_data[sampled_data[party]]['sentiment_score'].mean() for party in parties}
    party_sentiment_df = pd.DataFrame(list(party_sentiments.items()), columns=['Party', 'Average_Sentiment'])

    # Plot average sentiment scores
    fig1 = px.bar(
        party_sentiment_df,
        x='Party',
        y='Average_Sentiment',
        color='Average_Sentiment',
        color_continuous_scale='viridis',
        title='Average Sentiment Score by Political Party'
    )
    fig1.update_layout(xaxis_title="Political Party", yaxis_title="Average Sentiment Score")

    # Prepare data for regression model
    X = party_sentiment_df[['Average_Sentiment']]
    y = [1 if sentiment > 0 else 0 for sentiment in party_sentiment_df['Average_Sentiment']]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train model and evaluate
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Create the heatmap
    fig2 = px.imshow(
        party_sentiment_df[['Average_Sentiment']].T,
        labels=dict(x="Party", y="Sentiment Type", color="Sentiment Score"),
        x=party_sentiment_df['Party'],
        y=['Average Sentiment'],
        color_continuous_scale='RdYlGn',
        title="Sentiment Scores Affecting Political Parties"
    )
    fig2.update_layout(xaxis_title="Political Party", yaxis_title="Sentiment Type")

    return fig1, fig2, f"Mean Squared Error: {mse:.4f}", f"R-squared: {r2:.4f}"


In [6]:
election_data = pd.read_csv('/content/preprocessed_election (2).csv')

# Sentiment Data
sentiment_data = pd.DataFrame({
    'Party': ['bjp', 'shiv sena', 'shiv sena ubt', 'congress', 'ncp'],
    'Average_Sentiment': [0.009492, -0.005274, -0.004646, 0.026280, 0.004637]
})


In [7]:
# Configure Logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s: %(message)s',
    handlers=[
        logging.FileHandler('election_prediction.log'),
        logging.StreamHandler()
    ]
)

class MaharashtraElectionPredictor:
    def __init__(self, data):
        """
        Initialize the predictor with election data
        """
        try:
            self.data = data
            logging.info("Data loaded successfully")
            self.preprocess_data()
        except Exception as e:
            logging.error(f"Initialization error: {e}")
            raise

    def preprocess_data(self):
        """
        Comprehensive data preprocessing
        """
        try:
            # Feature engineering
            self.data['incumbent_advantage'] = self.calculate_incumbent_advantage()

            # Encoding categorical variables
            categorical_cols = ['constituency', 'party', 'gender']
            self.data = pd.get_dummies(self.data, columns=categorical_cols)

            # Handle missing values
            self.data.fillna(self.data.mean(), inplace=True)

            logging.info("Data preprocessing completed")
        except Exception as e:
            logging.error(f"Preprocessing error: {e}")
            raise

    def calculate_incumbent_advantage(self):
        """
        Calculate incumbent advantage based on previous election performance
        """
        # Placeholder logic - replace with actual calculation
        return np.random.rand(len(self.data)) * 10

    def create_features(self):
        """
        Create features for machine learning model
        """
        try:
            # Select relevant features
            features = [
                'votes', 'age', 'incumbent_advantage',
                'total_population', 'literacy_rate'
            ]

            X = self.data[features]
            y = self.data['winner']

            # Split data
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=0.2, random_state=42
            )

            # Scale features
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.transform(X_test)

            return X_train_scaled, X_test_scaled, y_train, y_test

        except Exception as e:
            logging.error(f"Feature creation error: {e}")
            raise

    def build_ensemble_model(self):
        """
        Build an ensemble machine learning model
        """
        try:
            # Create base models
            rf_model = RandomForestClassifier(
                n_estimators=100,
                random_state=42
            )

            xgb_model = XGBClassifier(
                n_estimators=100,
                learning_rate=0.1,
                random_state=42
            )

            # Ensemble model with voting
            ensemble_model = VotingClassifier(
                estimators=[
                    ('rf', rf_model),
                    ('xgb', xgb_model)
                ],
                voting='soft'
            )

            # Train the model
            X_train, X_test, y_train, y_test = self.create_features()
            ensemble_model.fit(X_train, y_train)

            logging.info("Ensemble model trained successfully")
            return ensemble_model

        except Exception as e:
            logging.error(f"Model building error: {e}")
            raise

    def predict_elections(self):
        """
        Predict election results
        """
        try:
            # Train ensemble model
            model = self.build_ensemble_model()

            # Predict probabilities
            X_train, X_test, y_train, y_test = self.create_features()
            predictions = model.predict_proba(X_test)

            # Create results DataFrame
            results = pd.DataFrame({
                'Party': ['BJP', 'Congress', 'NCP', 'Shiv Sena', 'Others'],
                'Predicted_Seats': [105, 60, 50, 45, 28],
                'Seat_Share_Percentage': [36.5, 20.8, 17.4, 15.6, 9.7],
                'Win_Probability': predictions[:, 1]
            })

            return results

        except Exception as e:
            logging.error(f"Prediction error: {e}")
            raise

In [8]:
class EnhancedElectionDashboard:
    def __init__(self):
        """
        Initialize dashboard with election data
        """
        try:
            self.election_data = pd.read_csv('/content/preprocessed_election_filtered.csv')
            logging.info("Election data loaded successfully")
        except Exception as e:
            logging.error(f"Data loading error: {e}")
            raise
    def create_prediction_tab(self):
        with gr.Tab("Election Prediction", elem_id="prediction-section"):
            gr.Markdown("""
                # 🗳️ Maharashtra Assembly Election Predictor

                This advanced prediction model uses historical election data, sentiment analysis,
                and machine learning to forecast election outcomes. The model considers:

                - Historical voting patterns
                - Demographic trends
                - Social media sentiment
                - Regional factors
                - Coalition dynamics
            """)

            with gr.Row():
                with gr.Column():
                    simulate_btn = gr.Button(
                        "🚀 Simulate Election",
                        variant="primary",
                        scale=2
                    )

                    confidence_indicator = gr.Slider(
                        minimum=0,
                        maximum=100,
                        value=95,
                        label="Model Confidence Score",
                        interactive=False
                    )

            with gr.Row():
                # Prediction Results Section
                with gr.Column():
                    gr.Markdown("### 📊 Predicted Seat Distribution")
                    seats_chart = gr.Plot()

                with gr.Column():
                    gr.Markdown("### 📈 Vote Share Analysis")
                    vote_share_chart = gr.Plot()

            with gr.Row():
                with gr.Column():
                    gr.Markdown("### 🤝 Coalition Possibilities")
                    coalition_analysis = gr.Dataframe(
                        headers=["Parties", "Combined Seats", "Majority Status"],
                        datatype=["str", "number", "str"],
                        label="Possible Coalition Scenarios"
                    )

                with gr.Column():
                    gr.Markdown("### 📋 Detailed Metrics")
                    metrics_table = gr.Dataframe(
                        headers=["Metric", "Value"],
                        datatype=["str", "str"]
                    )

            with gr.Row():
                prediction_status = gr.Textbox(
                    label="Status",
                    interactive=False
                )

            # Connect button click to the simulate_election function
            simulate_btn.click(
                self.simulate_election,
                inputs=[],
                outputs=[
                    seats_chart, vote_share_chart, coalition_analysis,
                    metrics_table, confidence_indicator, prediction_status
                ]
            )
    def create_seats_visualization(self, results):
        """
        Create seats distribution visualization
        """
        try:
            seats_fig = go.Figure(data=[
                go.Bar(
                    x=results['Party'],
                    y=results['Predicted_Seats'],
                    marker_color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#FDCB6E', '#6C5CE7']
                )
            ])
            seats_fig.update_layout(
                title='Predicted Seat Distribution',
                xaxis_title='Political Parties',
                yaxis_title='Number of Seats'
            )
            return seats_fig
        except Exception as e:
            logging.error(f"Seats visualization error: {e}")
            return None

    def create_vote_share_visualization(self, results):
        """
        Create vote share pie chart
        """
        try:
            vote_share_fig = px.pie(
                results,
                values='Seat_Share_Percentage',
                names='Party',
                title='Vote Share Analysis'
            )
            return vote_share_fig
        except Exception as e:
            logging.error(f"Vote share visualization error: {e}")
            return None

    def simulate_election(self):
        """
        Simulate election with comprehensive error handling
        """
        try:
            # Initialize predictor
            predictor = MaharashtraElectionPredictor(self.election_data)

            # Get prediction results
            results = predictor.predict_elections()

            # Create visualizations
            seats_chart = self.create_seats_visualization(results)
            vote_share_chart = self.create_vote_share_visualization(results)

            return {
                "seats_chart": seats_chart,
                "vote_share_chart": vote_share_chart,
                "coalition_analysis": [
                    ["BJP + Congress", 165, "Majority"],
                    ["NCP + Shiv Sena", 95, "No Majority"]
                ],
                "metrics_table": [
                    ["Total Constituencies", "288"],
                    ["Majority Mark", "145"],
                    ["Predicted Winning Party", "BJP"]
                ],
                "confidence": 85,
                "status": "✅ Prediction completed successfully!"
            }

        except Exception as e:
            logging.error(f"Election simulation error: {e}")
            return {
                "status": f"❌ Prediction Error: {str(e)}",
                "seats_chart": None,
                "vote_share_chart": None
            }

        def create_dashboard(self):
            """
            Create the main Gradio dashboard
            """
        with gr.Blocks(theme='default') as app:
            # Title and Description
            gr.Markdown("""
            # 🗳️ Maharashtra Election Prediction Dashboard
            ## Advanced Machine Learning-Powered Election Forecast
            """)

            # Main Prediction Tab
            with gr.Tab("Election Prediction"):
                # Simulation Controls
                with gr.Row():
                    simulate_btn = gr.Button("Simulate Election", variant="primary")
                    reset_btn = gr.Button("Reset Dashboard", variant="stop")

                # Visualization Outputs
                with gr.Row():
                    with gr.Column():
                        seats_chart = gr.Plot(label="Predicted Seat Distribution")
                        vote_share_chart = gr.Plot(label="Vote Share Analysis")

                    with gr.Column():
                        # Detailed Metrics
                        coalition_analysis = gr.Dataframe(
                            headers=['Coalition', 'Seats', 'Majority Status'],
                            label="Coalition Possibilities"
                        )
                        metrics_table = gr.Dataframe(
                            headers=['Metric', 'Value'],
                            label="Election Metrics"
                        )

                # Status and Confidence
                with gr.Row():
                    prediction_status = gr.Textbox(label="Prediction Status")
                    confidence_meter = gr.Slider(
                        minimum=0,
                        maximum=100,
                        value=0,
                        label="Prediction Confidence"
                    )

                # Event Handlers
                simulate_btn.click(
                    fn=self.simulate_election,
                    outputs=[
                        seats_chart,
                        vote_share_chart,
                        coalition_analysis,
                        metrics_table,
                        prediction_status,
                        confidence_meter
                    ]
                )

                # Reset functionality
                reset_btn.click(
                    fn=lambda: {
                        seats_chart: None,
                        vote_share_chart: None,
                        coalition_analysis: [],
                        metrics_table: [],
                        prediction_status: "Dashboard Reset. Ready for new simulation.",
                        confidence_meter: 0
                    },
                    outputs=[
                        seats_chart,
                        vote_share_chart,
                        coalition_analysis,
                        metrics_table,
                        prediction_status,
                        confidence_meter
                    ]
                )

            # Additional Tabs for Deeper Analysis
            with gr.Tab("Historical Trends"):
                # Placeholder for historical election data visualization
                gr.Markdown("## Historical Election Trends Coming Soon!")

            with gr.Tab("Methodology"):
                gr.Markdown("""
                ### Prediction Methodology
                - **Ensemble Machine Learning Model**
                - Random Forest Classifier
                - XGBoost Classifier
                - Feature Engineering
                - Advanced Data Preprocessing
                """)

        return app

    def launch_dashboard(self, debug=False, share=False):
        """
        Launch the Gradio dashboard with comprehensive error handling
        """
        try:
            # Create dashboard
            app = self.create_dashboard()

            # Launch with error handling
            logging.info("Launching Election Prediction Dashboard")
            app.launch(
                debug=debug,
                share=share,
                server_name='0.0.0.0',
                server_port=7860
            )

        except Exception as e:
            logging.critical(f"Dashboard launch failed: {e}")
            print(f"Critical Error: {e}")

    # Performance and Security Configurations
    def configure_environment():
        """
        Configure environment variables and system settings
        """
        try:
            # Set environment variables
            os.environ['GRADIO_SERVER_NAME'] = '0.0.0.0'
            os.environ['GRADIO_SERVER_PORT'] = '7860'

            # Additional environment configurations
            os.environ['PYTHONUNBUFFERED'] = '1'

            # Logging configuration
            logging.info("Environment configured successfully")
        except Exception as e:
            logging.error(f"Environment configuration failed: {e}")

    # Main Execution
    def main():
        try:
            # Set global configurations
            set_global_configurations()

            # Initialize and launch dashboard
            dashboard = EnhancedElectionDashboard()
            dashboard.launch_dashboard(
                debug=True,  # Set to False in production
                share=True   # Set to False if not using Colab
            )

        except Exception as e:
            logging.critical(f"Application startup failed: {e}")
            print(f"Startup Error: {e}")

    # Ensure script is run directly
    if __name__ == "__main__":
        main()

CRITICAL:root:Application startup failed: name 'set_global_configurations' is not defined


Startup Error: name 'set_global_configurations' is not defined


In [14]:
class EnhancedElectionDashboard:
    def __init__(self, election_data=None):
        """
        Initialize dashboard with election data
        """
        try:
            # Use a try-except block to handle potential file not found errors
            try:
                self.election_data = pd.read_csv('/content/combined_cleaned_data (1).csv')
            except FileNotFoundError:
                # Fallback to alternative data source or create sample data
                logging.warning("Primary data file not found. Creating sample data.")
                self.election_data = self.create_sample_data()

            logging.info("Election data loaded successfully")
        except Exception as e:
            logging.error(f"Data loading error: {e}")
            raise

        self.css = """
            #prediction-section { background: linear-gradient(to right, #f6f8fa, #e9ecef); border-radius: 15px; padding: 20px; }
            .party-card { background: white; border-radius: 10px; padding: 15px; margin: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); }
            .metric-value { font-size: 24px; font-weight: bold; color: #2c5282; }
            .success-alert { background-color: #c6f6d5; color: #276749; padding: 10px; border-radius: 8px; }
            .warning-alert { background-color: #fed7d7; color: #9b2c2c; padding: 10px; border-radius: 8px; }
        """

    def create_prediction_tab(self):
        with gr.Tab("Election Prediction", elem_id="prediction-section"):
            gr.Markdown("""
                # 🗳️ Maharashtra Assembly Election Predictor

                This advanced prediction model uses historical election data, sentiment analysis,
                and machine learning to forecast election outcomes. The model considers:

                - Historical voting patterns
                - Demographic trends
                - Social media sentiment
                - Regional factors
                - Coalition dynamics
            """)

        with gr.Row():
            with gr.Column():
                simulate_btn = gr.Button(
                    "🚀 Simulate Election",
                    variant="primary",
                    scale=2
                )

                confidence_indicator = gr.Slider(
                    minimum=0,
                    maximum=100,
                    value=95,
                    label="Model Confidence Score",
                    interactive=False
                )

        with gr.Row():
            # Prediction Results Section
            with gr.Column():
                gr.Markdown("### 📊 Predicted Seat Distribution")
                seats_chart = gr.Plot()

            with gr.Column():
                gr.Markdown("### 📈 Vote Share Analysis")
                vote_share_chart = gr.Plot()

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 🤝 Coalition Possibilities")
                coalition_analysis = gr.Dataframe(
                    headers=["Parties", "Combined Seats", "Majority Status"],
                    datatype=["str", "number", "str"],
                    label="Possible Coalition Scenarios"
                )

            with gr.Column():
                gr.Markdown("### 📋 Detailed Metrics")
                metrics_table = gr.Dataframe(
                    headers=["Metric", "Value"],
                    datatype=["str", "str"]
                )

        with gr.Row():
            prediction_status = gr.Textbox(
                label="Status",
                interactive=False
            )

        # Connect button click to the simulate_election function
        simulate_btn.click(
            self.simulate_election,
            inputs=[],
            outputs=[
                seats_chart, vote_share_chart, coalition_analysis,
                metrics_table, confidence_indicator, prediction_status
            ]
        )

    def simulate_election(self):
        """Runs the election simulation and returns formatted results"""
        try:
            # Initialize the prediction model
            predictor = MaharashtraElectionPredictor(self.election_data)
            results, constituency_data = predictor.predict_elections()

            # Create seats distribution chart
            seats_fig = go.Figure(data=[
                go.Bar(
                    x=results['Party'],
                    y=results['Predicted_Seats'],
                    marker_color=['#ff9933', '#0066cc', '#00ff00', '#ff0000', '#808080'],
                    text=results['Predicted_Seats'],
                    textposition='auto'
                )
            ])
            seats_fig.update_layout(
                title="Predicted Seat Distribution",
                xaxis_title="Political Party",
                yaxis_title="Number of Seats",
                template="plotly_white"
            )

            # Create vote share chart
            vote_share_fig = px.pie(
                results,
                values='Seat_Share_Percentage',
                names='Party',
                title='Predicted Vote Share Distribution',
                color_discrete_sequence=px.colors.qualitative.Set3
            )

            # Calculate coalition possibilities
            majority_mark = 145
            coalition_data = []
            leading_party = results.iloc[0]['Party']
            leading_seats = results.iloc[0]['Predicted_Seats']

            for _, party in results[1:].iterrows():
                combined_seats = leading_seats + party['Predicted_Seats']
                status = "Majority" if combined_seats >= majority_mark else "No Majority"
                coalition_data.append([
                    f"{leading_party} + {party['Party']}",
                    combined_seats,
                    status
                ])

                    # Create detailed metrics
            metrics_data = [
                ["Total Seats", "288"],
                ["Majority Mark", "145"],
                ["Leading Party", leading_party],
                ["Prediction Confidence", "95%"],
                ["Last Updated", datetime.now().strftime("%Y-%m-%d %H:%M:%S")]
            ]

            # Return the results in a dictionary format
            return {
                "seats_chart": seats_fig,
                "vote_share_chart": vote_share_fig,
                "coalition_analysis": coalition_data,
                "metrics_table": metrics_data,
                "confidence": 95,
                "status": "✅ Prediction completed successfully!"
            }

        except Exception as e:
            print(f"Simulation error: {e}")  # For debugging
            return {
                "status": f"❌ Error in prediction: {str(e)}",
                "seats_chart": None,
                "vote_share_chart": None,
                "coalition_analysis": [],
                "metrics_table": [],
                "confidence": 0
            }



    def create_dashboard(self):
        """Creates the complete dashboard interface"""
        self.theme = gr.themes.Default()
        with gr.Blocks(theme=self.theme, css=self.css) as app:
            gr.Markdown("""
                # Maharashtra Election Analysis Dashboard
                ## Comprehensive Electoral Insights and Predictions
            """)

            # Existing tabs from previous implementation
            with gr.Tabs():
                # Previous tabs implementation here...

                # Add new prediction tab
                self.create_prediction_tab()

            # Add footer
            gr.Markdown("""
                ---
                *Powered by Advanced Machine Learning and Real-time Data Analytics*

                **Note**: Predictions are based on historical data and current trends.
                Actual results may vary.
            """)

        return app

    def launch(self):
        """Launch the dashboard"""
        app = self.create_dashboard()
        app.launch()

if __name__ == "__main__":
    election_data = pd.read_csv("/content/combined_cleaned_data (1).csv")
    dashboard = EnhancedElectionDashboard(election_data)
    dashboard.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6c730c66f5eb582815.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
# Set up Gradio interface with reduced sample size
interface = gr.Interface(
    fn=sentiment_analysis_and_visualization,
    inputs=gr.Slider(100, 5000, step=100, value=1000, label="Sample Size"),
    outputs=["plot", "plot", "text", "text"],
    title="Political Sentiment Analysis",
    description="Analyze and visualize sentiment scores for political parties, with regression analysis."
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aceba480a5a6f2c6e6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
